# [실습1] RAG를 위한 Vector Score, Retriever

## 실습 목표
---
[실습2] 에서 시장 조사 문사 기반 QA 챗봇을 구성하기에 앞서, 문서를 저장하고 탐색하는 과정을 LangChain을 활용해 구현합니다.

## 실습 목차
---

1. **OllamaEmbeddings 생성:** 문서를 Vector로 변환하기 위한 OllamaEmbeddings를 생성합니다.

2. **벡터화된 문서 저장:** OllamaEmbeddings를 활용해 주어진 문서를 벡터로 변환하고, FAISS DB를 활용하여 저장합니다.

3. **Retriever Chain 구성:** 사용자의 입력과 가장 유사한 벡터화된 문서를 불러오는 Chain을 구성합니다.

4. **RAG Chain 구성:** 다음 실습에서 구현할 RAG 기반 챗봇의 일부 기능을 구현한 미니 RAG Chain을 구성해봅시다.

## 실습 개요
---
RAG 기반 챗봇의 핵심 구성 요소인 Vector Store, Retreiver를 구성해 봅시다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from operator import itemgetter

from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

- Ollama를 통해 Mistral 7B 모델을 불러옵니다. 모델을 다운로드 받는데는 약 3분 정도 소요됩니다.

In [2]:
!ollama pull mistral:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pull

## 1. OllamaEmbeddings 생성
- 문서를 Vector로 변환하기 위한 OllamaEmbeddings를 생성합니다.
- ChatOllama와 달리, Runnable하지 않습니다. 

Mistral 7B 모델을 사용하는 OllamaEmbeddings를 생성합니다. 

In [3]:
embeddings = OllamaEmbeddings(model="mistral:7b")

다음으로, 예시로 사용할 `Document`를 생성합니다.
- Document의 내용은 python 3.10 docs에서 발췌했습니다.

In [4]:
documents = [
    Document(
        page_content="random.seed(a=None, version=2) 난수 생성기를 초기화합니다. a가 생략되거나 None이면, 현재 시스템 시간이 사용됩니다. 운영 체제에서 임의성 소스(randomness sources)를 제공하면, 시스템 시간 대신 사용됩니다 (가용성에 대한 자세한 내용은 os.urandom() 함수를 참조하십시오).",
        metadata={"source": "random.seed"},
    ),
    Document(
        page_content="math.gcd(*integers) 지정된 정수 인자의 최대 공약수를 반환합니다. 인자 중 하나가 0이 아니면, 반환된 값은 모든 인자를 나누는 가장 큰 양의 정수입니다. 모든 인자가 0이면, 반환 값은 0입니다. 인자가 없는 gcd()는 0을 반환합니다.",
        metadata={"source": "math.gcd"},
    ),
    Document(
        page_content="re.search(pattern, string, flags=0) string을 통해 스캔하여 정규식 pattern이 일치하는 첫 번째 위치를 찾고, 대응하는 일치 객체를 반환합니다. 문자열의 어느 위치도 패턴과 일치하지 않으면 None을 반환합니다; 이것은 문자열의 어떤 지점에서 길이가 0인 일치를 찾는 것과는 다르다는 것에 유의하십시오.",
        metadata={"source": "re.search"},
    ),
    Document(
        page_content="copy.deepcopy(x[, memo]) x의 깊은 사본을 반환합니다.",
        metadata={"source": "copy.deepcopy"},
    )
]

## 2. 벡터화된 문서 저장
- OllamaEmbeddings를 활용해 주어진 문서를 벡터로 변환하고, FAISS DB를 활용하여 저장합니다.

In [5]:
vectorstore = FAISS.from_documents(
    documents,
    embedding=embeddings,
)

입력된 텍스트와 가장 유사한 document를 순서대로 보여주는 `similarity_search_with_score` 함수를 사용해봅시다.

In [6]:
vectorstore.similarity_search_with_score("파이썬에서 공약수를 구하는 방법")

[(Document(metadata={'source': 'math.gcd'}, page_content='math.gcd(*integers) 지정된 정수 인자의 최대 공약수를 반환합니다. 인자 중 하나가 0이 아니면, 반환된 값은 모든 인자를 나누는 가장 큰 양의 정수입니다. 모든 인자가 0이면, 반환 값은 0입니다. 인자가 없는 gcd()는 0을 반환합니다.'),
  149505.94),
 (Document(metadata={'source': 'random.seed'}, page_content='random.seed(a=None, version=2) 난수 생성기를 초기화합니다. a가 생략되거나 None이면, 현재 시스템 시간이 사용됩니다. 운영 체제에서 임의성 소스(randomness sources)를 제공하면, 시스템 시간 대신 사용됩니다 (가용성에 대한 자세한 내용은 os.urandom() 함수를 참조하십시오).'),
  168200.27),
 (Document(metadata={'source': 'copy.deepcopy'}, page_content='copy.deepcopy(x[, memo]) x의 깊은 사본을 반환합니다.'),
  168431.17),
 (Document(metadata={'source': 're.search'}, page_content='re.search(pattern, string, flags=0) string을 통해 스캔하여 정규식 pattern이 일치하는 첫 번째 위치를 찾고, 대응하는 일치 객체를 반환합니다. 문자열의 어느 위치도 패턴과 일치하지 않으면 None을 반환합니다; 이것은 문자열의 어떤 지점에서 길이가 0인 일치를 찾는 것과는 다르다는 것에 유의하십시오.'),
  173225.9)]

최대 공약수를 구하는 함수인 `math.gcd` 함수와 관련된 설명이 가장 유사한 Document라고 나타나는 것을 확인할 수 있습니다.

## 3. Retriever Chain 구성
- 사용자의 입력과 가장 유사한 벡터화된 문서를 불러오는 Chain을 구성합니다.

`langchain_community.vectorstores.FAISS` 데이터베이스는 "Runnable" 하지 않습니다. 이는 저희가 생성한 `vectorstore`를 그대로 활용하여 Chain을 구성할 수 없음을 의미합니다.

이를 해결하기 위해, `vectorstore`를 "Runnable"한 `Retriever` class로 변환하고, 이를 Chain에 연결해 볼 것입니다.

먼저, `Retriever` class로 변환한 `vectorstore` object를 `db_retriever`에 저장하고, `invoke()` 메서드를 사용할 수 있는지 확인해봅시다.

In [ ]:
db_retriever = vectorstore.as_retriever()
db_retriever.invoke("파이썬에서 최대 공약수를 구하는 방법")

잘 작동하는 것을 확인할 수 있습니다. 

## 4. 미니 RAG Chain 구성

이제 `db_retriever`를 활용해서 다음 실습에서 구현할 RAG 기반 챗봇의 일부 기능을 구현한 미니 RAG Chain을 구성해봅시다.

그전에, 이전 챕터에서 만들었던 Chain을 그대로 사용해서 똑같은 질문을 해봅시다.

In [ ]:
# role에는 "AI 어시스턴트"가, question에는 "당신을 소개해주세요."가 들어갈 수 있습니다.
messages_with_variables = [
    ("system", "당신은 {role} 입니다."),
    ("human", "{question}. 한글로 답하세요."),
]

prompt = ChatPromptTemplate.from_messages(messages_with_variables)
# 먼저, mistral:7b 모델을 사용하는 ChatOllama 객체를 생성합니다.
llm = ChatOllama(model="mistral:7b")

parser = StrOutputParser()

# pipe (|) 연산자를 통해 여러 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
# 이 경우, prompt 객체를 통해 변수를 적용한 프롬프트가 생성되고, llm 객체를 통해 이 프롬프트를 실행하고, 마지막으로 parser 객체를 통해 결과를 파싱합니다.
chain = prompt | llm | parser

In [ ]:
print(chain.invoke({"role": "친절한 페어 프로그래머", "question": "파이썬에서 최대 공약수를 구하는 방법."}))

저희가 사용한 LLM 모델이 학습한 데이터에는 파이썬 데이터도 포함되어 있기 때문에 별도로 Document를 추가하지 않더라도 잘 답변하는 것을 확인할 수 있습니다.

이제, 저희가 앞서 만든 `db_retriever` Retriever를 활용해서 Document를 기반으로 한 답변을 만들어 봅시다.

In [ ]:
# Retrieve한 문서 중 첫번째 문서를 가져오는 함수 정의
def get_first_doc(docs):
    return docs[0].page_content

# 시스템과 사용자 메시지를 포함한 프롬프트 템플릿 생성
messages_with_contexts = [
    ("system", "당신은 {role} 입니다. {context}를 레퍼런스로 답변해주세요."),
    ("human", "{question}. 한글로 답하세요."),
]

prompt_with_context = ChatPromptTemplate.from_messages(messages_with_contexts)

체인의 구성 요소를 만들었으니, 이제 이를 하나로 엮어서 체인으로 만들어 봅시다.
- 체인의 구성 요소 중 `itemgetter`는 딕셔너리에서 특정 키의 값을 가져오는 함수를 생성합니다.
- 즉, 사용자가 입력하는 딕셔너리에서 원하는 값을 추출하여 등록할 수 있도록 함수를 생성하여 Chain에 등록하는 과정입니다.

In [ ]:
# 체인 구성
# itemgetter는 딕셔너리에서 특정 키의 값을 가져오는 함수를 생성합니다.
# 즉, 사용자가 입력한 role과 question에 더해 
# context를 가져오는 체인을 활용해서 추출한 Document를 "context"에 넣어서 사용자에게 제공합니다.
qa_chain = (
    {"context": db_retriever | get_first_doc, "role": itemgetter("role"), "question": itemgetter("question")}
    | prompt_with_context
    | llm
    | StrOutputParser()
)

앞서 입력한 질문과 똑같은 질문을 다시 한번 입력해봅시다.

In [ ]:
print(qa_chain.invoke({"role": "친절한 페어 프로그래머", "question": "파이썬에서 공약수를 구하는 방법."}))

Context를 추가하지 않았을 때는 유클리드 호제법, `math.gcd()` 등 다양한 방법을 제시하는 데 반해, Context를 추가한 경우 Document에 있었던 `math.gcd()` 함수를 주로 답변하려는 경향을 보입니다.

### 추가 실습
- 위 문서에 없는 내용을 질문하고, 어떤 식으로 답변하는지 확인해보세요.

In [ ]:
# Python과 관련된 질문, 혹은 Topic을 벗어난 질문을 자유롭게 설정할 수 있습니다.
question = "파이썬에서 파일 입출력을 하는 방법을 설명해줘"

print(qa_chain.invoke({"role": "친절한 페어 프로그래머", "question": question}))

문서에 맞지 않는 질문을 할 때는 오히려 이상한 답변을 하는 경우도 있습니다. <br>
2일차 실습에서 이 경우에 대처하는 방법을 실습해 보겠습니다.

In [ ]:
# 아래 코드의 주석을 해제하고 실행하면 본 실습에서 다운받은 모델 파일을 삭제합니다.
# 각 실습에서 같은 모델이라도 다시 다운 받기 때문에, 
# 실습이 종료되었으면 아래 명령어를 실행하여 불필요한 파일을 삭제하는 것이 좋습니다.
# !rm -rf .ollama/models/*